In [24]:
from make_models import *
from helper_funcs import *

import numpy as np
import os 
from tqdm import tqdm
import pickle

In [2]:
df = pd.read_csv('../Data/ABIDE_df2.csv')
df = df.iloc[~pd.isna(df['bids_folder']).values]

In [3]:
data_dir = '../Assets/fc_mats_32smooth_new/'
files = [file for file in os.listdir(data_dir) if file.endswith('.npy')]
files.sort()
subs = df['bids_folder'].values
patients = df['DX_GROUP'].values==1

In [4]:
analysis_name = 'CVAE_2021-09-15 14:03:53.826090'

In [5]:
# CVAE WEIGHTS

batch_size = 32; latent_dim = 16
cvae, z_encoder, s_encoder, cvae_decoder = get_MRI_CCVAE_3D(input_shape=(32, 32, 32, 51),
                                                            latent_dim = latent_dim,
                                                            batch_size = batch_size, 
                                                            disentangle = True, 
                                                            gamma = 1,
                                                            kernel_size = 3,
                                                            filters = 64,
                                                            intermediate_dim = 128,
                                                            nlayers = 2,
                                                            learning_rate=0.001)

cvae.load_weights(os.path.join('../Assets/tf_weights/',analysis_name,'cvae_weights'))

In [6]:
nsamples = 10
nsubs = len(df)

In [7]:
bg_arr = np.zeros((nsamples,nsubs,latent_dim))
sl_arr = np.zeros((nsamples,nsubs,latent_dim))

In [21]:
for i in tqdm(range(nsamples),leave=True):
    for s in range(nsubs):
        arr = np.load(os.path.join(data_dir,subs[s]+'.npy'))
        arr = arr/2
        arr = arr[np.newaxis,:,:,:,:]

        vec_s = s_encoder.predict(arr)[2]
        vec_z = z_encoder.predict(arr)[2]

        bg_arr[i,s,:] = vec_z
        sl_arr[i,s,:] = vec_s

100%|██████████| 10/10 [14:09<00:00, 84.96s/it]


In [36]:
latent_vecs = dict()
latent_vecs['BG_ABIDE'] = bg_arr
latent_vecs['SL_ABIDE'] = sl_arr
latent_vecs['subs'] = subs

In [37]:
pickle.dump(latent_vecs,open(os.path.join('../Data/latent_vecs/','latent_vecs_'+analysis_name),'wb'))

In [ ]:
#cvae_decoder.predict(vec)

In [ ]:
#data_dir = '../Assets/fc_mats_32smooth_new'
#data_loader = cvae_data_loader(data_dir=data_dir, df=df, batch_size=32)